<div style="background-color:rgba(100, 160, 200, 0.33);">

<h1>Tutorial 2 - Transport Layer Protocols</h1>

<p>
Submission process:
</p>
<ul>
<li>Submission deadline is November 17, 14:00 CET (before the lecture) .</li>
<li>Commit and push your solution as single notebook file via git as <span style="font-family: monospace">./tutorial2/tutorial2.ipynb</span>. Please take care of the correct subfolder/filename since submission is denied otherwise.</li>
<li>During the first lecture after the deadline we will discuss a sample solution in class.</li>
<li>Afterwards, you have time until November 24, 14:00 CET (before the lecture) to submit a corrected version of your submission:</li>
<ol>
  <li>Rework your solution according to our discussion in class.</li>
  <li>Commit and push the corrected version as single file via git as <span style="font-family: monospace">./tutorial2/tutorial2.ipynb</span>. Please take care of the correct filename since submission is denied otherwise.</li>
</ol>
</ul>

<p>
Remarks:
</p>
<ul>
<li>Grading is done based on both versions of your submission.</li>
<li>If the first submission is missing your submission will not be graded.</li>
<li>If the second submission contains major flaws after revision not more than half of the credits for this tutorial can be achieved.</li>
<li>A sample solution is provided after November 24, 14:00 CET eventually.</li>
<li>
Please use <a href="mailto:acn@net.in.tum.de">acn@net.in.tum.de</a> for questions regarding lecture, tutorial, and project of ACN.
</li>
</ul>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<h3>Problem 1 TCP Congestion Control Fairness (4 credits)</h3>

<p>
This problem is focused on TCP congestion control. Different algorithms are observed and the fairness between them is evaluated.
</p>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
In the following we consider two TCP flows which share the same bottleneck link. The next cell visualizes the sending rates of the two flows.
</p>
</div>

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Download RTT CSV from ACN website
!wget -N https://acn.net.in.tum.de/exercise/tcp_cc.npz

# Load file into Python
data = np.load('tcp_cc.npz')
ts, flow1, flow2 = data['ts'], data['flow1'], data['flow2']

In [ ]:
# Helper function to plot data
def plot(*data, x_label=None, y_label=None, legends=[], y_min=None, y_max=None):
    plt.figure(figsize=(18, 6))
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    for xs, ys in data:
        plt.plot(xs, ys)
    plt.legend(legends)
    plt.gca().set_ylim(bottom=y_min, top=y_max)


plot((ts, flow1), (ts, flow2), x_label='Time in s', y_label='Sending Rate in bps', legends=['Flow1', 'Flow2'], y_min=0)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>a) [0.5 credits]</b>
One of the flows uses Cubic congestion control, the other one BBR. Identify which of the flows uses BBR. No credits without short reasoning.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>b) [0.5 credits]</b>
Write a function <span style="font-family: monospace">compute_sum()</span> which computes the total sending rate of two flows. It receives two lists and should return a list of the same size.
</p>

</div>

In [ ]:
def compute_sum(flow1, flow2):
    # begin insert code
    # end insert code

    return flow1

plot((ts, flow1), (ts, flow2), (ts, compute_sum(flow1, flow2)),
     x_label='Time in s', y_label='Sending Rate in bps', legends=['flow1', 'flow2', 'total'], y_min=0)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>c) [0.5 credits]</b>
Based on your results from b) estimate the bottleneck link's capacity.
What happens if the total sending rate of the two flows exceeds this value?
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>d) [0.5 credits]</b>
The minimum RTT of both flows is 50ms. Compute the path's BDP using the results from b) in kbit.
</p>

</div>

In [ ]:
BDP = 0
# begin insert code
# end insert code
print('BDP: {:.2f} kbit'.format(BDP))

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>e) [0.5 credits]</b>
In the following you will quantify the fairness of the two flows using 
<a href="https://tools.ietf.org/html/rfc5166#section-2.3.1">Jain's Fairness Index</a>. Explain two advantages of this index.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>f) [1 credits]</b>
Write a function <span style="font-family: monospace">compute_fairness()</span> which receives a list of equal sized lists an then computes Jain's Index elementwise.
</p>

</div>

In [ ]:
# Example:
# input: [[1, 2, 3], [4, 5, 6]]
# output: [jain(1,4), jain(2, 5), jain(3, 6)]
# with jain(x1, x2, ...): jain index of x1, x2, ...

def compute_fairness(*lists):
    
    # this checks is all input lists have same size
    assert len(set(map(len, lists))) == 1
    
    # begin insert code
    # end insert code

    # Dummy output
    # returns [1,1, ...] of the same length as the first parameter
    return [1, ] * len(lists[0])

plot((ts, compute_fairness(flow1, flow2)), x_label='Time in s', y_label='Jain\'s Index', y_min=0.5, y_max=1)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>g) [0.5 credits]</b>
Considering the results from e), assess the fairness between Cubic and BBR. Also take the results from the next cell into account (assuming you implemented the <span style="font-family: monospace">compute_fairness</span> function).
</p>

</div>

In [ ]:
print('Flow1 transmitted {:.2f} Mbit/s on average.'.format(np.mean(flow1) / 10**6))
print('Flow2 transmitted {:.2f} Mbit/s on average.'.format(np.mean(flow2) / 10**6))

total_fairness = compute_fairness([np.mean(flow1)], [np.mean(flow2)])
print('This results in a total fairness of {:.5f}'.format(total_fairness[0]))

<div style="background-color:rgba(100, 160, 200, 0.33);">
<h3>Problem 2 TCP Round-trip-time Estimation (3 credits)</h3>

<p>
This problem is focused on TCP Timestamps Options.
</p>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>a) [1 credits]</b>
    TCP can use Timestamps Options to estimate the RTT of a connection. 
    Have a look at <a href="https://tools.ietf.org/html/rfc7323">RFC 7323</a> to understand how TCP Timestamps Options work.
    The options contains two fields:
</p>
<ul>
    <li>TS Value (TSval)</li>
    <li>TS Echo Reply (TSecr).</li>
</ul>

<p>
    Briefly explain (in your own words) how these values are used to estimate the RTT. 
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
In the following you will write code which computes RTT estimations from TCP Timestamp Options.
Keep in mind that this time we are a passive observer of the connection and do not have any information about the internal TCP state of any of the endpoints. This means we cannot rely on the pseudo-timestamp values contained in the options. 
</p>
</div>

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Download RTT CSV from ACN website
!wget -N https://acn.net.in.tum.de/exercise/tcp_timestamp.npz

# Load file into Python
data = np.load('tcp_timestamp.npz')
flow1_tsval, flow1_tsecr = data['flow1_tsval'], data['flow1_tsecr']
flow2_tsval, flow2_tsecr = data['flow2_tsval'], data['flow2_tsecr']
flow3_tsval, flow3_tsecr = data['flow3_tsval'], data['flow3_tsecr']
flow4_tsval, flow4_tsecr = data['flow4_tsval'], data['flow4_tsecr']

In [ ]:
# Helper function to plot data
def plot(*data, x_label=None, y_label=None, legends=[], y_min=None, y_max=None):
    plt.figure(figsize=(18, 6))
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    for xs, ys in data:
        plt.plot(xs, ys)
    plt.legend(legends)
    plt.gca().set_ylim(bottom=y_min, top=y_max)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>b) [1 credits]</b>
Write a function <span style="font-family: monospace">compute_rtt()</span> which receives as input two lists:
    <ul>
    <li>tsval: contains tuples with (timestamp, tsval) which contains all tsval sent and the according timestamp.</li>
    <li>tsecr: contains tuples with (timestamp, tsecr) which contains all tsecr received and the according timestamp.</li>
    </ul>
    You can assume that the values are only captured on the sending side of the connection.
    
The function should return a list of tuples containing (timestamp, rtt).
</p>

</div>

In [ ]:
def compute_rtt(ts_val, ts_ecr):
    # begin insert code
    # end insert code

    return [(0, 0)]

plot(zip(*compute_rtt(flow1_tsval, flow1_tsecr)), x_label='Time in s', y_label='RTT in s', legends=['Flow1'], y_min=0)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>c) [1 credits]</b>
The dataset contains four TCP flows using different congestion control algorithms.
For each flow, try to find out which algorithm was used and explain your reasoning.
You do not get any credits without an explanation.
</p>

</div>

In [ ]:
plot(zip(*compute_rtt(flow1_tsval, flow1_tsecr)),
     zip(*compute_rtt(flow2_tsval, flow2_tsecr)),
     zip(*compute_rtt(flow3_tsval, flow3_tsecr)),
     zip(*compute_rtt(flow4_tsval, flow4_tsecr)),
     x_label='Time in s', y_label='RTT in s',
     legends=['Flow1', 'Flow2', 'Flow3', 'Flow4'], y_min=0)

<div style="background-color:rgba(100, 160, 200, 0.33);">
<h3>Problem 3 Exponential Weighted Moving Average (3 credits)</h3>

<p>
In this exercise you will compute the retransmission timout of a TCP connection. The first cell downloads a csv file which contains RTT samples of a TCP connection. You will then analyze the RTT values and finally compute the retransmission timout.
</p>
</div>

In [ ]:
%matplotlib inline

import numpy as np
import matplotlib.pyplot as plt

# Download RTT CSV from ACN website
!wget -N https://acn.net.in.tum.de/exercise/tcp_rtt.npz

# Load file into Python
data = np.load('tcp_rtt.npz')
ts, rtt = data['ts'], data['rtt']

In [ ]:
# Helper function to plot data
def plot(*data, x_label=None, y_label=None, legends=[], y_min=None, y_max=None):
    plt.figure(figsize=(18, 6))
    plt.xlabel(x_label)
    plt.ylabel(y_label)
    for xs, ys in data:
        plt.plot(xs, ys)
    plt.legend(legends)
    plt.gca().set_ylim(bottom=y_min, top=y_max)


plot((ts, rtt), x_label='Time in s', y_label='RTT in ms', legends=['flow1'])

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>a) [0.1 credits]</b>
Have a look at the above graph. Which congestion control algorithm was used by the TCP sender? (No explanation required)
</p>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>b) [0.5 credits]</b>
Write a function to compute the exponential weighted moving average.
The function <span style="font-family: monospace">ewma()</span> gets two parameters, a list of values and a value alpha as weight for the new samples.
</p>
</div>

In [ ]:
def ewma(values, alpha):
    # begin insert code
    # end insert code
    return values

smoothed_rtt = ewma(rtt, 0.125)
plot((ts, rtt), (ts, smoothed_rtt), x_label='Time in s', y_label='RTT in ms', legends=['RTT', 'EWMA'])


<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>c) [0.4 credits]</b>
Compare the results when using different values for alpha and explain how it impacts the resulting average.
</p>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>d) [1.5 credits]</b>
    Have a look at <a href="https://tools.ietf.org/html/rfc6298">RFC 6298</a> how TCP computes the retransmission timeout (RTO). In the following we implement a function to compute this value for our connection. We will ignore the clock granularity G, e.g. consider it small enough.
</p>

<p>
Write the function <span style="font-family: monospace">compute_rto()</span> which receives a list of RTT samples and computes the RTO values using the two state variables SRTT (smoothed round-trip time) and RTTVAR (round-trip time variation) as explained in <a href="https://tools.ietf.org/html/rfc6298">RFC 6298</a>.
</p>
</div>

In [ ]:
def compute_rto(samples):
    # constants
    ALPHA = 0.125
    BETA = 0.25
    K = 4
    G = 0

    # begin insert code
    # end insert code
    return samples

plot((ts, rtt), (ts, compute_rto(rtt)), x_label='Time in s', y_label='RTT in ms', legends=['RTT', 'RTO'])

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>e) [0.5 credits]</b>
Discuss the influence on TCP if the RTT is considerably overestimated or underestimated.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<h3>Problem 4 QUIC (3 credits)</h3>

<p>
This problem is about the QUIC protocol.
QUIC is considered as the sucessor for the TCP/TLS stack and is the based for the new HTTP/3 standard.
</p>
</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>a) [0.5 credits]</b>
Name all protocols which are usually (e.g. HTTP/1.1) used on top of IP when you visit <a href="https://acn.net.in.tum.de/">https://acn.net.in.tum.de/</a>.
Which protocols will be used when you would visit the same page with HTTP/3?
</p>

</div>

<b>HTTP/1.1:</b>
<ul>
</ul>

<b>HTTP/3:</b>
<ul>
</ul>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
    The QUIC protocol is currently standardized by the IETF.
    You can find the RFC <a href="https://www.rfc-editor.org/rfc/rfc9000.html">here</a>.
    For the next questions you need to have a closer look into the draft.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>b) [0.5 credits]</b>
    QUIC differenciates between packets and frames.
    Name <b>all</b> packet types available in QUIC.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>c) [0.5 credits]</b>
    Name <b>5</b> frame types specified in the RFC.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
    Qlog is a logging format for QUIC. The next cell downloads a qlog file of QUIC connection which transferred a small file between two hosts. 
</p>

</div>

In [ ]:
import json

# Download qlog file
!wget -N https://acn.net.in.tum.de/exercise/quic.qlog

# Load file into Python
with open('quic.qlog') as f:
    data = json.load(f)

print('\nThe qlog is formatted as follows:')
print(data.keys())

print('\nThis qlog contains {} traces.'.format(len(data['traces'])))
print('Traces is a list containing the following values:')
print(data['traces'][0].keys())

events = data['traces'][0]['events']
print('\nEach trace containes a list of events:')
print(events[0])

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
    For the following questions you can either parse the qlog file in Python, or use the qvis tool which nicely visualizes the qlog. We recommend the later.
    You can use the version hosted at <a href="https://qvis.edm.uhasselt.be/">https://qvis.edm.uhasselt.be/</a>.
</p>
<p>
    Also, the format of the file is specified in <a href="https://tools.ietf.org/html/draft-marx-qlog-main-schema-01">this draft</a>.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>d) [0.5 credits]</b>
    Which QUIC version is used in this connection. Paste the version ID as well as which version is specified by it (<a href="https://github.com/quicwg/base-drafts/wiki/QUIC-Versions">hint</a>). <br>
    Also, find out which QUIC implementation was used to generate the qlog file.
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
    The qlog containes HTTP/3 request response pair. The request can be found in event 23.
</p>

</div>

In [ ]:
import pprint
pprint.pprint(events[23])

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>e) [0.5 credits]</b>
    <ul>
        <li>Name the packet type included in this event and explain why this packet type has to be used.</li>
        <li>Name <b>and</b> briefly explain all frame types in this event.</li>
    <ul>
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p>
<b>f) [0.5 credits]</b>
    <ul>
        <li>Which event carries the response of the server?</li>
        <li>How large is the file requested by the client?</li>
    <ul>
</p>

</div>

<div style="background-color:rgba(100, 160, 200, 0.33);">
<p><b>Advanced Computer Networking by Prof. Dr.-Ing. Georg Carle</b></p>
<p>Teaching assistants: Sebastian Gallenmüller, Benedikt Jaeger, Max Helm, Patrick Sattler, Johannes Zirngibl</p>
</div>